# CIFAR-10 転移学習

In [0]:
from keras.datasets import cifar10

import numpy as np
import tensorflow as tf
import keras

from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization

random_state = 42

In [0]:
(X_train_cifar,y_train_cifar),(X_test_cifar,y_test_cifar)=cifar10.load_data()

In [0]:
X_train = X_train_cifar.astype('float32')
X_test = X_test_cifar.astype('float32')

In [0]:
from keras.utils import to_categorical

y_train_cifar = to_categorical(y_train_cifar)
y_test_cifar = to_categorical(y_test_cifar)

In [0]:
X_test_cifar.shape

In [0]:
X_train = X_train_cifar[:40000]
y_train = y_train_cifar[:40000]

X_val = X_train_cifar[40000:]
y_val = y_train_cifar[40000:]

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, Input, Lambda
from keras.callbacks import EarlyStopping

In [0]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Input

# 事前学銃済みモデルの作成
base_model = VGG16(weights='imagenet', include_top=False)

In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
base_model_output = base_model.output
add_model = GlobalAveragePooling2D()(base_model_output)
add_model = Dense(1024, activation='relu')(add_model)
add_model = Dense(10, activation='softmax')(add_model)

# 転移学習モデル
model = Model(inputs=base_model.input, outputs=add_model)

print(model.summary())

In [0]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [0]:
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit(x=X_train, y=y_train, batch_size=128, epochs=10, verbose=1,
          validation_data=(X_val, y_val), callbacks=[early_stopping])